In [ ]:
import os

from astropy.time import Time, TimeDelta
import numpy as np
import pandas as pd
import yaml

from lsst_efd_client import EfdClient

In [ ]:
ifilename = "m1m3_hardpointdata.yaml"
efd_name = "ncsa_teststand_efd"

In [ ]:
with open(ifilename) as ifile:
    info = yaml.safe_load(ifile)
try:
    info['expectedRate'] = eval(info['expectedRate'])
except TypeError:
    pass

In [ ]:
client = EfdClient(efd_name)

In [ ]:
time_increment = TimeDelta(600, format='sec', scale='tai')
start_time = Time("2020-05-30T00:00:00", scale='tai')
base_end_time = Time("2020-06-01T12:00:00", scale='tai')
expected_num_messages = round(time_increment.sec) * info['expectedRate']

end_time = base_end_time + time_increment

df = await client.select_time_series(info['tablename'], info['columnName'], start_time, end_time)

In [ ]:
df1 = df[info['columnName']].resample(time_increment.datetime).count().to_frame()

In [ ]:
df1.rename(columns={info['columnName']: "Received Messages"}, inplace=True)

In [ ]:
df1["Expected Messages"] = np.full((df1.shape[0],), expected_num_messages)
df1["Percent Loss"] = (1.0 - df1["Received Messages"] / df1["Expected Messages"]) * 100.0

In [ ]:
df1.head()